In [1]:
import sys
import logging
import brittle_wit as bw
import aiohttp
import asyncio
import gzip
from brittle_wit import streaming_api
from brittle_wit.executors import LOGGER
from brittle_wit.streaming import basic_streamer

In [2]:
LOGGER.setLevel(logging.DEBUG)

ch = logging.StreamHandler(sys.stdout)
ch.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
ch.setFormatter(formatter)
LOGGER.addHandler(ch)

In [3]:
help(streaming_api.statuses.sample)

Help on function sample in module brittle_wit.streaming_api.statuses:

sample(*, delimited=ELIDE, stall_warnings=ELIDE)
    Returns a small random sample of all public statuses. The Tweets returned
    by the default access level are the same, so if two different clients
    connect to this endpoint, they will see the same Tweets.
    
    :param delimited: Specifies whether messages should be length-delimited.
        See delimited or more information.
    
    :param stall_warnings: Specifies whether stall warnings should be
        delivered. See stall_warnings for more information.



In [4]:
APP_CRED = bw.AppCredentials.load_from_env()
CLIENT_CRED = bw.ClientCredentials.load_from_env()

In [5]:
import time
output_path = "/Users/generativist/Downloads/Trump/{}.jsonl.gz".format(time.time())
fp = gzip.open(output_path, "wb")

In [6]:
n_total, n_trump = 0, 0

def quantify_trump_infection(msg):
    global n_total, n_trump, fp
    if b'trump' in msg.lower():
        n_trump += 1
            
    n_total += 1
    if n_total % 10000 == 0 and n_total > 0:
        print(n_total, n_trump, n_trump/n_total)
    
    fp.write(msg + b"\n")

In [7]:
stream = basic_streamer(APP_CRED, CLIENT_CRED, 
                        streaming_api.statuses.sample(),
                        quantify_trump_infection,
                        False)
asyncio.get_event_loop().run_until_complete(stream)

10000 172 0.0172
20000 381 0.01905
30000 565 0.018833333333333334
40000 732 0.0183
50000 890 0.0178
60000 1052 0.017533333333333335
70000 1197 0.0171
80000 1346 0.016825
90000 1535 0.017055555555555556
100000 1741 0.01741
110000 1914 0.0174
120000 2115 0.017625
130000 2290 0.017615384615384616
140000 2474 0.01767142857142857
150000 2648 0.017653333333333333
160000 2720 0.017
170000 2816 0.01656470588235294
180000 2917 0.016205555555555556
190000 3014 0.015863157894736843
200000 3101 0.015505
210000 3187 0.015176190476190477
220000 3261 0.014822727272727272
230000 3344 0.014539130434782608
240000 3434 0.014308333333333333
250000 3521 0.014084
260000 3632 0.013969230769230769
270000 3759 0.013922222222222223
280000 3851 0.01375357142857143
290000 3958 0.013648275862068966
300000 4046 0.013486666666666666
310000 4153 0.013396774193548388
320000 4254 0.01329375
330000 4377 0.013263636363636363
340000 4502 0.013241176470588235
350000 4621 0.013202857142857143
360000 4743 0.013175
370000 485

KeyboardInterrupt: 

In [ ]:
fp.close()